In [1]:
import os
import re
from pathlib import Path
from shutil import copyfile
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
csv_folder_name = 'csv_data_hate'
total_samples = 3000

In [3]:
df_count = pd.DataFrame(columns=['topic', 'count', 'proportion_count'])

csv_dfs = {}
total_count = 0

entries = os.listdir(csv_folder_name)
regex = re.compile('([a-zA-Z0-9_-]*)_final\.csv')

for filename in entries:
    if filename.startswith('.'): continue
    matched = regex.search(filename)
    if matched:
        topic_name = matched.group(1)
        csv_path = os.path.join(csv_folder_name, filename)
        csv_dfs[topic_name] = pd.read_csv(csv_path, index_col=0)
        row_count = len(csv_dfs[topic_name].index)
        total_count += row_count
        df_count.loc[len(df_count)] = [topic_name, row_count, -1]
        print('{:<20}{:<7}'.format(topic_name, row_count))


immigration_2021-04 186    
immigration_2020-02 183    
mask_04             16     
politics_12         2883   
immigration_2021-02 39     
mask_08             91     
mask_2021-02        53     
politics_2021-02    576    
immigration_12      52     
vaccine_12          228    
politics_2021-04    263    
politics_2020-02    2706   
asianhate_12        399    
ageism_12           537    
mask_2021-04        117    
mask_2020-02        2      
immigration_08      144    
vaccine_08          436    
ageism_2021-02      237    
mask_12             63     
asianhate_08        815    
politics_04         4617   
ageism_08           709    
ageism_2020-02      117    
ageism_2021-04      691    
vaccine_2020-02     94     
vaccine_2021-04     166    
asianhate_2021-02   285    
vaccine_04          475    
immigration_04      93     
vaccine_2021-02     112    
ageism_04           1600   
asianhate_2021-04   323    
asianhate_2020-02   538    
asianhate_04        2887   
politics_08         

In [4]:
df_count['minimum'] = df_count['count'].apply(lambda x: x if x <= 20 else 20)
df_count['subtract'] = df_count['count'] - df_count['minimum']
min_sum = df_count['minimum'].sum()
df_count['proportion_count'] = df_count['subtract'].apply(lambda x: round((x/(total_count - min_sum))*(total_samples - min_sum)))
df_count['proportion_count'] += df_count['minimum']

if df_count['proportion_count'].sum() > total_samples:
    diff = df_count['proportion_count'].sum() - total_samples
    df_count.at[df_count.proportion_count.idxmax(), 'proportion_count'] -= diff
elif df_count['proportion_count'].sum() < total_samples:
    diff = total_samples - df_count['proportion_count'].sum()
    df_count.at[df_count.proportion_count.idxmax(), 'proportion_count'] += diff

df_count

,topic,count,proportion_count,minimum,subtract
0,immigration_2021-04,186,33,20,166
1,immigration_2020-02,183,33,20,163
2,mask_04,16,16,16,0
3,politics_12,2883,242,20,2863
4,immigration_2021-02,39,21,20,19
5,mask_08,91,26,20,71
6,mask_2021-02,53,23,20,33
7,politics_2021-02,576,63,20,556
8,immigration_12,52,22,20,32
9,vaccine_12,228,36,20,208


In [ ]:
# df_count['proportion_count'] = df_count['count'].apply(lambda x: round((x/total_count)*total_samples))
# df_count['proportion_count'] = round(total_samples / len(df_count.index))
# df_count.proportion_count[df_count.topic=='mask_08'] += 1
# df_count

In [5]:
df_count.to_csv(csv_folder_name + '/sample_count.csv', index=False)
df_count['proportion_count'].sum()

3100

In [6]:
df_samples = pd.DataFrame(columns=['topic', 'hashtag', 'tweet_id', 'image_path', 'sample_path', 'body_text', 'image_text'])

for _, row in df_count.iterrows():
    topic_name = row['topic']
    df_topic = csv_dfs[topic_name].sample(n=row['proportion_count'], random_state=42)
    df_topic.insert(loc=0, column='topic', value=topic_name)
    df_topic.insert(loc=4, column='sample_path', value=np.nan)
    df_topic.insert(loc=len(df_topic.columns), column='image_text', value=np.nan)
    df_samples = df_samples.append(df_topic)

df_samples = df_samples.sample(frac=1, random_state=42).reset_index(drop=True)
df_samples.to_csv(csv_folder_name + '/sample_data.csv', index=False)
df_samples

,topic,hashtag,tweet_id,image_path,sample_path,body_text,image_text,text_with_OCR
0,asianhate_04,ChinaVirus,1254162507020763137,./tweet_data/asianhate_04/ChinaVirus/tweets4/E...,NaN,@Reuters How long can we stay at home? This is...,NaN,sy virus guard sustainable disinfection 10 day...
1,vaccine_04,BillGatesBioTerrorist,1248344434120560642,./tweet_data/vaccine_04/BillGatesBioTerrorist/...,NaN,So. Many. SHEEPLE! WAKE UUUPP!\n#BillGatesBioT...,NaN,ie alright
2,politics_08,TrumpIsALaughingStock,1294450197435645952,./tweet_data/politics_08/TrumpIsALaughingStock...,NaN,@AintParler @Lorrain86857415 @TheWorldWatch @S...,NaN,oa boohoo sorry aaa wa del ah
3,politics_08,TrumpIsANationalDisgrace,1292919661206687748,./tweet_data/politics_08/TrumpIsANationalDisgr...,NaN,Gallup poll: 4 out of 5 Republicans don't thin...,NaN,republican satisfaction state america plummet ...
4,politics_04,TrumpLiesPeopleDie,1248647765044776961,./tweet_data/politics_04/TrumpLiesPeopleDie/tw...,NaN,@catjung @AmoneyResists Keep believing the bul...,NaN,dont recognize pea ae es ta ws mt aor
...,...,...,...,...,...,...,...,...
3095,vaccine_12,antivax,1338931589456285696,./tweet_data/vaccine_12/antivax/tweets3/EpTXIL...,NaN,@bekindhavehope @WhiteHouse @realDonaldTrump W...,NaN,ea apa aa aa vd obama bush clinton vaccine tv
3096,politics_04,Trump2020Landslide,1248286193281835009,./tweet_data/politics_04/Trump2020Landslide/tw...,NaN,Nice surprise from mom when she dropped off so...,NaN,ik ally great hc ase
3097,politics_04,TrumpIsALaughingStock,1253817479693918208,./tweet_data/politics_04/TrumpIsALaughingStock...,NaN,He couldn't take questions at today's press br...,NaN,wo alll fa trump recommend crit nt die ot
3098,politics_04,TrumpIsALaughingStock,1253887775411253251,./tweet_data/politics_04/TrumpIsALaughingStock...,NaN,This just in...#TrumpIsALaughingStock #lockdow...,NaN,ak sete copyright


# Please only run cells below

## Copy selected samples and generate sample path

### Before running this part, please check that the topics you want to process is NOT in the annotation_data folder

In [7]:
import os
from pathlib import Path
from shutil import copyfile
import pandas as pd
import numpy as np

In [8]:
# df_count = pd.read_csv(csv_folder_name + '/sample_count.csv')
df_samples = pd.read_csv(csv_folder_name + '/sample_data.csv')
annot_data_path = './annot_data_hate'
Path(annot_data_path).mkdir(exist_ok=True)

In [9]:
def copy_image_set_path(img_path):
    image_name = img_path.split('/')[-1]
    dst_path = os.path.join(annot_data_path, image_name)
    copyfile(img_path, dst_path)
    return dst_path

In [10]:
df_samples['sample_path'] = df_samples['image_path'].apply(copy_image_set_path)
df_samples

,topic,hashtag,tweet_id,image_path,sample_path,body_text,image_text,text_with_OCR
0,asianhate_04,ChinaVirus,1254162507020763137,./tweet_data/asianhate_04/ChinaVirus/tweets4/E...,./annot_data_hate/EWeuJgbXkAAz1OD.jpg,@Reuters How long can we stay at home? This is...,NaN,sy virus guard sustainable disinfection 10 day...
1,vaccine_04,BillGatesBioTerrorist,1248344434120560642,./tweet_data/vaccine_04/BillGatesBioTerrorist/...,./annot_data_hate/EVMCpJxXQAEsSLi.jpg,So. Many. SHEEPLE! WAKE UUUPP!\n#BillGatesBioT...,NaN,ie alright
2,politics_08,TrumpIsALaughingStock,1294450197435645952,./tweet_data/politics_08/TrumpIsALaughingStock...,./annot_data_hate/EfbPlPOWAAEj3NC.jpg,@AintParler @Lorrain86857415 @TheWorldWatch @S...,NaN,oa boohoo sorry aaa wa del ah
3,politics_08,TrumpIsANationalDisgrace,1292919661206687748,./tweet_data/politics_08/TrumpIsANationalDisgr...,./annot_data_hate/EfFfVjwX0AUmmcQ.jpg,Gallup poll: 4 out of 5 Republicans don't thin...,NaN,republican satisfaction state america plummet ...
4,politics_04,TrumpLiesPeopleDie,1248647765044776961,./tweet_data/politics_04/TrumpLiesPeopleDie/tw...,./annot_data_hate/EVQWgf-UcAA1Z3K.jpg,@catjung @AmoneyResists Keep believing the bul...,NaN,dont recognize pea ae es ta ws mt aor
...,...,...,...,...,...,...,...,...
3095,vaccine_12,antivax,1338931589456285696,./tweet_data/vaccine_12/antivax/tweets3/EpTXIL...,./annot_data_hate/EpTXILlXIAEYPwt.png,@bekindhavehope @WhiteHouse @realDonaldTrump W...,NaN,ea apa aa aa vd obama bush clinton vaccine tv
3096,politics_04,Trump2020Landslide,1248286193281835009,./tweet_data/politics_04/Trump2020Landslide/tw...,./annot_data_hate/EVLNqynUEAEN_TT.jpg,Nice surprise from mom when she dropped off so...,NaN,ik ally great hc ase
3097,politics_04,TrumpIsALaughingStock,1253817479693918208,./tweet_data/politics_04/TrumpIsALaughingStock...,./annot_data_hate/EWZ0KwQUwAARojT.jpg,He couldn't take questions at today's press br...,NaN,wo alll fa trump recommend crit nt die ot
3098,politics_04,TrumpIsALaughingStock,1253887775411253251,./tweet_data/politics_04/TrumpIsALaughingStock...,./annot_data_hate/EWa0SOrXQAAlLsp.jpg,This just in...#TrumpIsALaughingStock #lockdow...,NaN,ak sete copyright


In [11]:
df_samples.to_csv(csv_folder_name + '/sample_data.csv', index=False)

### Old code below, don't run

In [3]:
def get_sample_path_and_copy(df, topic, annot_data_path):
    # if the sample_path already exists
    if not pd.isna(df['sample_path']):
        return df['sample_path']

    # if topic is not the selected topic
    if pd.isna(df['sample_path']) and df['topic'] != topic:
        return np.nan
    
    image_name = df['image_path'].split('/')[-1]
    dst_path = os.path.join(annot_data_path, image_name)
    copyfile(df['image_path'], dst_path)
    return dst_path

In [4]:
sample_folder = './annotation_data'
Path(sample_folder).mkdir(exist_ok=True)

for _, row in df_count.iterrows():
    topic_name = row['topic']
    topic_data_path = os.path.join('./', 'data_' + topic_name)
    annot_data_path = os.path.join(sample_folder, topic_name)
    # if data_{topic_name} folder doesn't exist
    if not Path(topic_data_path).is_dir():
        print('{} not exists; pass this topic'.format(topic_data_path))
        continue

    # if the sampled data for this topic already exists
    if Path(annot_data_path).is_dir():
        print('{} already exist; pass this topic'.format(annot_data_path))
        continue

    Path(annot_data_path).mkdir()

    df_samples['sample_path'] = df_samples.apply(get_sample_path_and_copy, axis=1, topic=topic_name, annot_data_path=annot_data_path)

    print('Topic \"{}\" processed'.format(topic_name))

Topic "Vaccine" processed
./data_AsianHate not exists; pass this topic
./data_Boomer not exists; pass this topic
./data_Political not exists; pass this topic
./data_Mask not exists; pass this topic
Topic "Immigration" processed


In [5]:
df_samples.to_csv('./csv_data/sample_data.csv', index=False)
df_samples

,topic,hashtag,tweet_id,image_path,sample_path,body_text,image_text,text_with_OCR
0,Immigration,refugees,1297606828461883394,./data_Immigration/refugees/tweets4/EgIGg6aXsA...,./annotation_data/Immigration/EgIGg6aXsAAX5-Y.jpg,Just one week left to get one of these hand em...,NaN,oe
1,Mask,maskup,1297309098669510656,./data_Mask/maskup/tweets4/EgD3vYTUcAE_JMC.jpg,NaN,🚧 #SFSO keeping our citizens updated on evacua...,NaN,er ha an ey st ge at Bra wy yo ae ae ay at oes...
2,Boomer,caresact,1291464418715459589,./data_Boomer/caresact/tweets1/Eew0B6PWAAYZPCX...,NaN,"Arguably, this one segment from the #CARESAct ...",NaN,aS is he
3,Mask,NoMasks,1290561571312328706,./data_Mask/NoMasks/tweets1/Eej-523WsAAJa6h.jpg,NaN,@RupertaMargate @FrancaisFarage @John35542712 ...,NaN,Ser ah
4,Immigration,cdnpoli,1297693936782213123,./data_Immigration/cdnpoli/tweets4/EgJVvpkUMAA...,./annotation_data/Immigration/EgJVvpkUMAALB8Z.png,"You beat me to it, lol .. #cpc #cpcldr #cdnpol...",NaN,You organize two car funeral procession iS
...,...,...,...,...,...,...,...,...
2395,Mask,NoMasks,1291460379638484994,./data_Mask/NoMasks/tweets1/EewwXv5WkAEb91v.jpg,NaN,What's this...a new wave of virus???\n\n#KBF #...,NaN,Let me introduce you to our next problem Seen ...
2396,Boomer,trumpliesamericansdie,1290437319611342851,./data_Boomer/trumpliesamericansdie/tweets1/Ee...,NaN,@realDonaldTrump 1st-To be so proud of a large...,NaN,reed la HI WA ha ant aa Trump to draw out at l...
2397,Boomer,caresact,1293904775055052801,./data_Boomer/caresact/tweets2/EfTfiMrWsAA35Y4...,NaN,Ready to re-open? Use your #CARESAct Funding t...,NaN,YOUR GO HERE Oe WEVE GOT vol
2398,Political,Trump2020,1291952240534728704,./data_Political/Trump2020/tweets2/Ee3vtmRX0AA...,NaN,@HowDoYouMakeAU2 @Jenny06980536 @TerriSm600279...,NaN,al LEAVE NEVER LEAVE
